# Developing Alternatives to VaR (Value-at-Risk)

| Name            | Email                          |
|-----------------|--------------------------------|
| Rohit Singh     | rohitkumar.singh7885@gmail.com |
| Sahil Shinde    | sahil311292@gmail.com          |
| Shantanu Mishra | 8hantanu@gmail.com             |

## Pre-requisites

### Install packages

In [92]:
!pip install yfinance
!pip install pandas_datareader


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python -m pip install --upgrade pip


### Imports

In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

## Import data 
Data from year 2000 to 2020 for treasury yields for 1, 5, and 10 years

### US treasury bonds rates for 1 year yields

In [94]:
treasury_1y = pdr.get_data_yahoo('^IRX', start='2010-01-01', end='2020-12-31')
treasury_1y.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,0.080,0.080,0.055,0.055,0.055,0
2010-01-05,0.060,0.065,0.040,0.060,0.060,0
2010-01-06,0.060,0.060,0.045,0.045,0.045,0
2010-01-07,0.045,0.055,0.040,0.045,0.045,0
2010-01-08,0.045,0.045,0.035,0.040,0.040,0


### US treasury bonds rates for 5 years yields

In [95]:
treasury_5y = pdr.get_data_yahoo('^FVX', start='2010-01-01', end='2020-12-31')
treasury_5y.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,2.684,2.684,2.633,2.652,2.652,0
2010-01-05,2.583,2.593,2.549,2.558,2.558,0
2010-01-06,2.566,2.625,2.558,2.573,2.573,0
2010-01-07,2.625,2.642,2.574,2.600,2.600,0
2010-01-08,2.642,2.654,2.527,2.566,2.566,0


### US treasury bonds rates for 10 years yields

In [96]:
treasury_10y = pdr.get_data_yahoo('^TNX', start='2010-01-01', end='2020-12-31')
treasury_10y.head()


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,3.859,3.859,3.808,3.841,3.841,0
2010-01-05,3.790,3.800,3.749,3.755,3.755,0
2010-01-06,3.771,3.837,3.761,3.808,3.808,0
2010-01-07,3.845,3.859,3.800,3.822,3.822,0
2010-01-08,3.843,3.851,3.775,3.808,3.808,0


### Calculate the daily returns

In [97]:
# Calculate daily returns for treasury yields
treasury_1y['daily_return'] = treasury_1y['Adj Close'].pct_change()
treasury_5y['daily_return'] = treasury_5y['Adj Close'].pct_change()
treasury_10y['daily_return'] = treasury_10y['Adj Close'].pct_change()

# Drop missing values
treasury_1y.dropna(inplace=True)
treasury_5y.dropna(inplace=True)
treasury_10y.dropna(inplace=True)



## Metrics

### Value at Risk (VaR)

In [98]:
def calculate_var(df, confidence_level=0.99):
    """
    Calculate Value at Risk (VaR) for a given confidence level and time horizon
    """
    
    sorted_returns = df.sort_values('daily_return')
    var = -sorted_returns['daily_return'].quantile(1-confidence_level)

    return var

# Calculate VaR round for 1 year treasury yield with 99% confidence level
var_1y = calculate_var(treasury_1y, 0.99)
print(f'Value at Risk for 1 year treasury yield with 99% confidence level is {var_1y:.3f}')

# Calculate VaR for 5 year treasury yield with 99% confidence level
var_5y = calculate_var(treasury_5y, 0.99)
print(f'Value at Risk for 5 year treasury yield with 99% confidence level is {var_5y:.3f}')

# Calculate VaR for 10 year treasury yield with 99% confidence level
var_10y = calculate_var(treasury_10y, 0.99)
print(f'Value at Risk for 10 year treasury yield with 99% confidence level is {var_10y:.3f}')


Value at Risk for 1 year treasury yield with 99% confidence level is 0.698
Value at Risk for 5 year treasury yield with 99% confidence level is 0.105
Value at Risk for 10 year treasury yield with 99% confidence level is 0.071


### Expected Shortfall (ES)

In [99]:
# Compute ES for a 10 day liquidity horizon based on Basel IV norms which uses a 97.5% confidence level
def calculate_es(df, confidence_level=0.975, time_horizon=10):
    """
    Calculate Expected Shortfall (ES) for a given confidence level and time horizon
    """
    
    sorted_returns = df.sort_values('daily_return')
    var = sorted_returns['daily_return'].quantile(1-confidence_level)
    es = -sorted_returns[sorted_returns['daily_return'] <= var]['daily_return'].mean()
    
    return es

# Calculate ES for 1 year treasury yield with 97.5% confidence level
es_1y = calculate_es(treasury_1y, 0.975)
print(f'Expected Shortfall for 1 year treasury yield with 97.5% confidence level is {es_1y:.3f}')

# Calculate ES for 5 year treasury yield with 97.5% confidence level
es_5y = calculate_es(treasury_5y, 0.975)
print(f'Expected Shortfall for 5 year treasury yield with 97.5% confidence level is {es_5y:.3f}')

# Calculate ES for 10 year treasury yield with 97.5% confidence level
es_10y = calculate_es(treasury_10y, 0.975)
print(f'Expected Shortfall for 10 year treasury yield with 97.5% confidence level is {es_10y:.3f}')


Expected Shortfall for 1 year treasury yield with 97.5% confidence level is 0.765
Expected Shortfall for 5 year treasury yield with 97.5% confidence level is 0.113
Expected Shortfall for 10 year treasury yield with 97.5% confidence level is 0.082


### ES based on Differentiated Liquidity Horizon (ESD)

In [100]:
def calculate_es_with_liquidity_horizon(df, confidence_level=0.975):

    time_horizon = [10, 20, 40, 60, 120]
    es_arr = []
    es_scaled_arr = []

    for i, horizon in enumerate(time_horizon):
        es = calculate_es(df, confidence_level, horizon)
        if i != 0:
            es_scaled = es * np.sqrt((time_horizon[i] - time_horizon[i-1]) / 10)
        else:
            es_scaled = es
        es_arr.append(es)
        es_scaled_arr.append(es_scaled)

    es_b = np.sqrt(np.sum(np.square(es_scaled_arr)))

    return es_b

# Calculate ESD for 1 year treasury yield with 97.5% confidence level
esd_1y = calculate_es_with_liquidity_horizon(treasury_1y, 0.975)
print(f'Expected Shortfall for 1 year treasury yield with 97.5% confidence level is {esd_1y:.3f}')

# Calculate ESD for 5 year treasury yield with 97.5% confidence level
esd_5y = calculate_es_with_liquidity_horizon(treasury_5y, 0.975)
print(f'Expected Shortfall for 5 year treasury yield with 97.5% confidence level is {esd_5y:.3f}')

# Calculate ESD for 10 year treasury yield with 97.5% confidence level
esd_10y = calculate_es_with_liquidity_horizon(treasury_10y, 0.975)
print(f'Expected Shortfall for 10 year treasury yield with 97.5% confidence level is {esd_10y:.3f}')


Expected Shortfall for 1 year treasury yield with 97.5% confidence level is 2.650
Expected Shortfall for 5 year treasury yield with 97.5% confidence level is 0.393
Expected Shortfall for 10 year treasury yield with 97.5% confidence level is 0.285


### ES based on Spectral Risk Measure (ESS)

In [101]:
# TODO: